In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 00:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)


2025-03-03 18:47:57,257 INFO: Initializing external client
2025-03-03 18:47:57,257 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 18:47:57,840 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215656


In [6]:
df= fg.select_all().read()

2025-03-03 18:48:54,869 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 205, in __init__
    self._health_check()
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/retrying.py", line 266, in call
    raise attempt.get()
          ^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/retrying.py", line 301, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/six.py", line 724, in reraise
    raise value
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/ret

2025-03-03 18:48:59,881 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 364, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 427, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/taxi_project/lib/python3.11/site-packages/retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/taxi_project/lib/pytho

FeatureStoreException: Could not read data using Hopsworks Feature Query Service.

In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,138,179.0,2025-03-03 21:00:00+00:00
1,78,0.0,2025-03-03 21:00:00+00:00
2,107,65.0,2025-03-03 21:00:00+00:00
3,221,0.0,2025-03-03 21:00:00+00:00
4,38,0.0,2025-03-03 21:00:00+00:00
...,...,...,...
246,207,0.0,2025-03-03 21:00:00+00:00
247,168,1.0,2025-03-03 21:00:00+00:00
248,202,0.0,2025-03-03 21:00:00+00:00
249,191,0.0,2025-03-03 21:00:00+00:00


In [ ]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,138,179.0,2025-03-03 21:00:00+00:00
1,78,0.0,2025-03-03 21:00:00+00:00
2,107,65.0,2025-03-03 21:00:00+00:00
3,221,0.0,2025-03-03 21:00:00+00:00
4,38,0.0,2025-03-03 21:00:00+00:00
...,...,...,...
246,207,0.0,2025-03-03 21:00:00+00:00
247,168,1.0,2025-03-03 21:00:00+00:00
248,202,0.0,2025-03-03 21:00:00+00:00
249,191,0.0,2025-03-03 21:00:00+00:00


In [9]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-03 18:47:36,329 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-03-03 18:47:36,332 INFO: Initializing external client
2025-03-03 18:47:36,332 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 18:47:36,962 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215656
Current UTC time: 2025-03-03 23:47:36.329601+00:00
Next hour: 2025-03-04 00:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 00:00
Found 0 records


In [10]:
now = datetime.now(timezone.utc)

In [ ]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  251 non-null    int32                  
 1   predicted_demand    251 non-null    float64                
 2   pickup_hour         251 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.0 KB


In [13]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [14]:
dt = current_date.ceil('h')


In [15]:
#Note: this was causing error
# fs = get_feature_store()  
# fg = fs.get_feature_group(  
#     name=config.FEATURE_GROUP_MODEL_PREDICTION,  
#     version=1  
# )  
# query = fg.select_all().read().head(5)
# # query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
# query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
# results = query.read()
#write_options={"use_spark": True}

In [16]:
#Note: you commented this line
#fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read().head(251)

In [17]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [18]:
predictions = fetch_next_hour_predictions()

2025-03-03 18:47:37,921 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 18:47:37,924 INFO: Initializing external client
2025-03-03 18:47:37,924 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 18:47:38,512 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215656
Current UTC time: 2025-03-03 23:47:37.921750+00:00
Next hour: 2025-03-04 00:00:00+00:00
Found 0 records


In [ ]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-03 18:47:39,231 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-03-03 18:47:39,234 INFO: Initializing external client
2025-03-03 18:47:39,234 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 18:47:39,818 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215656


In [20]:
# this was commented out
#df = fg.select_all().read().head(251)

In [21]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,138,179.0,2025-03-03 21:00:00+00:00
1,78,0.0,2025-03-03 21:00:00+00:00
2,107,65.0,2025-03-03 21:00:00+00:00
3,221,0.0,2025-03-03 21:00:00+00:00
4,38,0.0,2025-03-03 21:00:00+00:00
...,...,...,...
246,207,0.0,2025-03-03 21:00:00+00:00
247,168,1.0,2025-03-03 21:00:00+00:00
248,202,0.0,2025-03-03 21:00:00+00:00
249,191,0.0,2025-03-03 21:00:00+00:00


In [22]:
df["pickup_hour"].max()

Timestamp('2025-03-03 21:00:00+0000', tz='Etc/UTC')

In [23]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 00:00:00+00:00


In [24]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
